In [1]:
!pip install --upgrade torch datasets transformers pandas tensorflow > /dev/null 2>&1
!pip install --upgrade wandb simpletransformers > /dev/null 2>&1 || pip install --user --upgrade wandb simpletransformers > /dev/null 2>&1

In [3]:
# Import daily_dialog dataset
from datasets import load_dataset
daily_dialog = load_dataset("daily_dialog")

# Processor
def process_row(batch):    
    result = {}
    text = result["text"] = []
    labels = result["labels"] = []
    
    acts_batch = batch["act"]
    dialog_batch = batch["dialog"]
    
    for row_idx in range(len(dialog_batch)):
        
        acts = acts_batch[row_idx]
        dialog = dialog_batch[row_idx]
        num = len(dialog)
        assert num == len(acts)

        for idx in range(num):
            text.append(dialog[idx])
            labels.append(acts[idx])
    
    return result

# Process dataset
mapped = daily_dialog.map(process_row, batched=True, remove_columns=['dialog', 'act', 'emotion'], load_from_cache_file=True).flatten()

# Split model into train, dev etc.
train_df = mapped['train'].to_pandas()
eval_df = mapped['validation'].to_pandas()

Using custom data configuration default
Reusing dataset daily_dialog (/home/jupyter/.cache/huggingface/datasets/daily_dialog/default/1.0.0/c03444008e9508b8b76f1f6793742d37d5e5f83364f8d573c2747bff435ea55c)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [4]:
# Set up wandb
import wandb

key = "2ee46b01a610d015968a7abe18a895014b237fbc"
project = "daily-dialog"
entity = "diwank"

wandb.login(key=key)
wandb.init(project=project, entity=entity)


# Turn off debug logs
import transformers
transformers.logging.set_verbosity_error()


# model configuration
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)

model_args = ClassificationArgs(
    n_gpu=2,
    num_train_epochs=12, 
    learning_rate=2e-5,
    model_type="deberta", 
    model_name="microsoft/deberta-base",
    wandb_project=project,
    use_multiprocessing=False,
    train_batch_size=24,
    evaluate_during_training=True,
    use_early_stopping=False,
    early_stopping_patience=12,
    scheduler="polynomial_decay_schedule_with_warmup",
)


# Create a ClassificationModel
model = ClassificationModel("deberta", "microsoft/deberta-base", args=model_args, num_labels=5)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: diwank (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


In [6]:
# Train model
model.train_model(train_df, eval_df=eval_df)

Epoch:   0%|          | 0/12 [00:00<?, ?it/s]

Running Epoch 0 of 12:   0%|          | 0/3633 [00:00<?, ?it/s]

/home/jupyter/.local/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


KeyboardInterrupt: 

In [ ]:
!gsutil cp -r ./outputs gs://internal.whitehead.ai/daily-dialog/large-`date '+%m-%d-%Y_%H_%M_%S'`/outputs

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    eval_df
)